In [1]:
from pdf2image import convert_from_path
from PyPDF2 import PdfFileReader
import cv2
import pytesseract
import os

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [3]:
# code for converting pdf to images and identifying tables from the last 2 pages to save into the target csv file
for filename in os.listdir("pdf_tables"):
    file = "pdf_tables/" + filename
    pdf_len = PdfFileReader(open(file,'rb')).getNumPages()
    # identifying two last pages in every pdf file
    pages = convert_from_path(file, 500, first_page = pdf_len - 1, 
                              poppler_path = r"C:\Users\mspya\Desktop\MSBA\Hackathon\poppler-22.01.0\Library\bin")
    k = 1
    for page in pages:
        page.save('aux.jpg', 'JPEG')
        img_cv = cv2.imread('aux.jpg')
        # rotating the image of the page before last
        if k == 1:
            img_cv = cv2.rotate(img_cv, cv2.cv2.ROTATE_90_CLOCKWISE)
        # resizing the image
        img_resized = cv2.resize(img_cv,
                         (int(img_cv.shape[1] + (img_cv.shape[1] * .1)),
                          int(img_cv.shape[0] + (img_cv.shape[0] * .25))),
                         interpolation=cv2.INTER_AREA)
        # changing the color scheme to grey scale
        img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
        # reading the data from the image into text file
        data = pytesseract.image_to_string(img_rgb)
        # cleaning the data
        try:
            i1 = data.index("TOTAL")
            i2 = data.index("As reported to")
            output = data[i1:i2]
            for s in ["°", "?", "®", "*", '"', "(", ")", ","]:
                output = output.replace(s, "")
            text = output.split("\n")
            i = 0
            while i < len(text):
                while True:
                    ind = text[i].find(" ")
                    if ind == len(text[i])-1:
                        break
                    if text[i].find(" ") == -1:
                        break
                    if not text[i][ind+1].isdecimal():
                        text[i] = text[i].replace(" ", "", 1)
                    else:
                        break
                i += 1
            text = [el for el in text if el != ""]
            output = "\n".join(text)
            output = output.replace(" ", ",")
            # saving the output as csv file
            output_file = "csv_tables/" + filename.replace(".pdf", "") + '_' + str(k) + '.csv'
            with open(output_file, 'w') as f:
                f.write(output)
            k += 1
        except:
            pass